In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!sudo apt-get install libsqlite3-dev
!git clone https://github.com/mapbox/tippecanoe.git && cd tippecanoe && make -j && make install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=076cc25b46e7e06b5a0e34d906500a03f9a4cd2ca431f4c3fa58aa23dd142742
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok



Suggested packages:
  sqlite3-doc
The following NEW packages will be installed:
  libsqlite3-dev
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 697 kB of archives.
After this operation, 2372 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libsqlite3-dev amd64 3.31.1-4ubuntu0.5 [697 kB]
Fetched 697 kB in 1s (604 kB/s)
Selecting previously unselected package libsqlite3-dev:amd64.
(Reading database ... 107763 files and directories currently installed.)
Preparing to unpack .../libsqlite3-dev_3.31.

In [2]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
import mercantile
import csv
import json

In [3]:
csv_files = [
    "/kaggle/input/new-york-city-transport-statistics/mta_1706.csv",
    "/kaggle/input/new-york-city-transport-statistics/mta_1708.csv",
    "/kaggle/input/new-york-city-transport-statistics/mta_1710.csv",
    "/kaggle/input/new-york-city-transport-statistics/mta_1712.csv",
]

usecols=["RecordedAtTime", "VehicleLocation.Latitude", "VehicleLocation.Longitude", "PublishedLineName", "VehicleRef", "ArrivalProximityText"]

rename_columns = {
    'RecordedAtTime': 'time',
    'VehicleLocation.Latitude': 'lat',
    'VehicleLocation.Longitude': 'long',
    "PublishedLineName": "linename",
    "VehicleRef": "ref",
    "ArrivalProximityText": "text",
}

In [4]:
%%time
for csv_file in csv_files:
    df = pd.read_csv(csv_file, usecols=usecols)
    df = df.rename(columns=rename_columns)
    df.fillna("NaN", inplace=True)
    
    features = []
    
    # for _, row in df.head(10000).iterrows():
    for _, row in df.iterrows():
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [float(row['long']), float(row['lat'])]
            },
            "properties": {
                "time": row['time'],
                "linename": row['linename'],
                "ref": row['ref'],
                "text": row['text'],
            }
        }
        features.append(feature)
        
    file_name = "/kaggle/working/" + csv_file.split("/")[-1].replace(".csv", ".geojson")
    with open(file_name, 'w') as geojson_file:
        json.dump(features, geojson_file, indent=2)
    
print('Done!')

Done!
CPU times: user 1h 3min 20s, sys: 1min 19s, total: 1h 4min 40s
Wall time: 1h 5min 27s


In [5]:
!tippecanoe -zg --layer=map --output-to-directory=./outputs --no-tile-compression --drop-densest-as-needed --extend-zooms-if-still-dropping --force -P *.geojson

mta_1706.geojson:3: Found ] at top level
mta_1706.geojson:4: Found ] at top level
mta_1706.geojson:2: Found colon at top level
mta_1706.geojson:26924983: Reached EOF without all containers being closed
In JSON object [{"type":"Feature","geometry":{"type":"Point","coordinates":[-73.923861]}}]
mta_1708.geojson:2: Found colon at top level
mta_1708.geojson:2: Found colon at top level
mta_1708.geojson:2: Found colon at top level
mta_1708.geojson:25854402: Reached EOF without all containers being closed
In JSON object [{}]
mta_1710.geojson:2: Found } at top level
mta_1710.geojson:3: Found ] at top level
mta_1710.geojson:4: Found ] at top level
mta_1710.geojson:27465142: Reached EOF without all containers being closed
In JSON object [{"type":"Feature","geometry":{"type":"Point","coordinates":[]}}]
mta_1712.geojson:2: Found colon at top level
mta_1712.geojson:4: Found ] at top level
mta_1712.geojson:2: Found colon at top level
mta_1712.geojson:25847324: Reached EOF without all containers being

In [6]:
# import uvicorn
# from fastapi import FastAPI
# from fastapi.staticfiles import StaticFiles
# from fastapi.middleware.cors import CORSMiddleware
# from pyngrok import ngrok
# import nest_asyncio
# import threading


# app = FastAPI()

# # middlewares
# app.add_middleware(
#     CORSMiddleware, # https://fastapi.tiangolo.com/tutorial/cors/
#     allow_origins=['*'], # wildcard to allow all, more here - https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Origin
#     allow_credentials=True, # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Credentials
#     allow_methods=['*'], # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Methods
#     allow_headers=['*'], # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Headers
# )
# app.mount("/", StaticFiles(directory="/kaggle/working/outputs"), name="outputs")

# # specify a port
# port = 8000
# ngrok_tunnel = ngrok.connect(port)

# # where we can visit our fastAPI app
# print('Public URL:', ngrok_tunnel.public_url)


# nest_asyncio.apply()

# # finally run the app
# uvicorn.run(app, port=port)